### Setup of the env

Install and import of python libraries 

In [1]:
!pip3 install selenium
!pip3 install pandas

In [2]:
import requests
import pprint
import pandas as pd
import time
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import selenium
import json

Set the following variables to download data:
- sleep_time: to be polite with TEDx (number of seconds between different request)

In [3]:
sleep_time = 0.1
log = True

This notebook uses Chrome Driver to simulate user interaction with TEDx.
To set up Chrome Driver on your laptop please refer to https://chromedriver.chromium.org/downloads


In [4]:
chromedriver_path =  '/Users/feder/Downloads/chromedriver/chromedriver'

In [5]:
def get_browser():
    chrome_options = wd.ChromeOptions()
    chrome_options.add_argument('log-level=3')
    browser = wd.Chrome(chromedriver_path, options=chrome_options)
    return browser

browser = get_browser()

# Import TEDx data


In [6]:
df = pd.read_csv("tedx_dataset.csv") 
my_tedx_list = df.to_dict('records')
len(my_tedx_list)

4467

In [7]:
skip_sentences = ['(Applause)', '(Laughter)', '(Laughs)', '(Inaudible)']

def get_transcript(my_tedx):
    if log:
        print("Current url: " + my_tedx['url'])
    
    try:
        browser.get(my_tedx['url'] + '/transcript')
        if browser.title == "TED | 404: Not Found":
            raise Exception('Transcript not available')
        
        # ensures all timestamps are generated
        try:
            player = WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "video[src]")))
        except:
            pass
        
        # ensures English transcript is selected
        browser.find_element_by_xpath("//select[@name='transcript']/option[@value='en']").click()
    
        l = browser.find_elements_by_css_selector(".Grid.Grid--with-gutter.d\:f\@md.p-b\:4")
    
        transcript = []
        for rel in l:
            sentence = rel.find_elements_by_css_selector("p")[0].text
            # skips useless lines
            if sentence.strip() not in skip_sentences:  
                timestamp = rel.find_elements_by_css_selector("button.sb")[0].text
                transcript.append({"timestamp": timestamp, "sentence": sentence})
        my_tedx['transcript'] = transcript
    except Exception as err:
        print(err)
        my_tedx['transcript'] = {}

    return my_tedx

In [ ]:
my_tedx_list_final = []
for my_tedx in my_tedx_list:
    my_tedx_list_final.append(get_transcript(my_tedx))
    
print("Done")

Current url: https://www.ted.com/talks/alexandra_auer_the_intangible_effects_of_walls_apr_2020
Current url: https://www.ted.com/talks/elizabeth_gilbert_it_s_ok_to_feel_overwhelmed_here_s_what_to_do_next
Current url: https://www.ted.com/talks/butterscotch_accept_who_i_am
Transcript not available
Current url: https://www.ted.com/talks/ethan_lisi_what_it_s_really_like_to_have_autism
Current url: https://www.ted.com/talks/daniel_finkel_can_you_solve_the_sea_monster_riddle
Current url: https://www.ted.com/talks/matt_walker_why_sleep_matters_now_more_than_ever
Transcript not available
Current url: https://www.ted.com/talks/sonia_shah_how_to_make_pandemics_optional_not_inevitable
Transcript not available
Current url: https://www.ted.com/talks/mary_ellen_hannibal_how_you_can_help_save_the_monarch_butterfly_and_the_planet
Current url: https://www.ted.com/talks/todd_dufresne_history_vs_sigmund_freud
Current url: https://www.ted.com/talks/rabbi_lord_jonathan_sacks_how_we_can_navigate_the_coronavi

Current url: https://www.ted.com/talks/yves_behar_how_surfboards_connect_us_to_nature
Current url: https://www.ted.com/talks/jon_gray_the_power_of_the_afro_pick
Current url: https://www.ted.com/talks/chip_kidd_why_books_are_here_to_stay
Current url: https://www.ted.com/talks/jennifer_8_lee_why_1_5_billion_people_eat_with_chopsticks
Current url: https://www.ted.com/talks/debbie_millman_the_function_and_fashion_of_eyeglasses
Current url: https://www.ted.com/talks/david_biello_how_the_compass_unlocked_the_world
Current url: https://www.ted.com/talks/a_j_jacobs_the_evolution_of_the_coffee_cup_lid
Current url: https://www.ted.com/talks/paola_antonelli_why_pasta_comes_in_all_shapes_and_sizes
Current url: https://www.ted.com/talks/debbie_millman_how_symbols_and_brands_shape_our_humanity
Current url: https://www.ted.com/talks/raghuram_rajan_how_to_revive_and_empower_local_communities
Transcript not available
Current url: https://www.ted.com/talks/noelle_martin_online_predators_spread_fake_porn

Current url: https://www.ted.com/talks/tom_nash_the_perks_of_being_a_pirate_feb_2019
Current url: https://www.ted.com/talks/alex_rosenthal_the_artists_think_like_a_coder_ep_5
Current url: https://www.ted.com/talks/suzanne_lee_why_biofabrication_is_the_next_industrial_revolution
Current url: https://www.ted.com/talks/marco_tempest_a_swarm_of_mini_drones_makes_magic
Current url: https://www.ted.com/talks/alex_gendler_can_you_solve_the_dragon_jousting_riddle
Current url: https://www.ted.com/talks/colette_pichon_battle_climate_change_will_displace_millions_here_s_how_we_prepare
Current url: https://www.ted.com/talks/anna_piperal_what_a_digital_government_looks_like
Current url: https://www.ted.com/talks/eden_girma_the_mysterious_life_and_death_of_rasputin
Current url: https://www.ted.com/talks/emma_schachner_the_secret_weapon_that_let_dinosaurs_take_over_the_planet
Current url: https://www.ted.com/talks/leon_marchal_the_urgent_case_for_antibiotic_free_animals
Current url: https://www.ted.c

Current url: https://www.ted.com/talks/peter_beck_small_rockets_are_the_next_space_revolution
Current url: https://www.ted.com/talks/celine_valery_the_dangers_of_mixing_drugs
Current url: https://www.ted.com/talks/mani_vajipey_how_india_s_local_recyclers_could_solve_plastic_pollution
Current url: https://www.ted.com/talks/dorsa_amir_why_do_humans_have_a_third_eyelid
Current url: https://www.ted.com/talks/rabiaa_el_garani_hope_and_justice_for_women_who_ve_survived_isis
Current url: https://www.ted.com/talks/kaitlyn_sadtler_and_heather_j_faust_why_haven_t_we_cured_arthritis
Current url: https://www.ted.com/talks/amma_y_ghartey_tagoe_kootin_a_historical_musical_that_examines_black_identity_in_the_1901_world_s_fair
Current url: https://www.ted.com/talks/ma_yansong_urban_architecture_inspired_by_mountains_clouds_and_volcanoes
Current url: https://www.ted.com/talks/eli_pariser_what_obligation_do_social_media_platforms_have_to_the_greater_good
Current url: https://www.ted.com/talks/martha_red

Current url: https://www.ted.com/talks/herman_narula_the_transformative_power_of_video_games
Current url: https://www.ted.com/talks/marlene_daut_the_first_and_last_king_of_haiti
Current url: https://www.ted.com/talks/tara_djokic_this_ancient_rock_is_changing_our_theory_on_the_origin_of_life
Current url: https://www.ted.com/talks/lucy_hone_the_three_secrets_of_resilient_people
Transcript not available
Current url: https://www.ted.com/talks/paul_rucker_criminalization_of_survival
Transcript not available
Current url: https://www.ted.com/talks/efosa_ojomo_reducing_corruption_takes_a_specific_kind_of_investment
Current url: https://www.ted.com/talks/david_peterson_why_language_is_humanity_s_greatest_invention
Current url: https://www.ted.com/talks/aracelis_girmay_for_estefani_third_grade_who_made_me_a_card
Current url: https://www.ted.com/talks/camilla_arndal_andersen_what_happens_in_your_brain_when_you_taste_food
Current url: https://www.ted.com/talks/mitchell_katz_what_the_us_health_care

...

Current url: https://www.ted.com/talks/leland_melvin_an_astronaut_s_story_of_curiosity_perspective_and_change
Current url: https://www.ted.com/talks/emily_quinn_the_way_we_think_about_biological_sex_is_wrong
Current url: https://www.ted.com/talks/kasha_patel_the_benefits_of_using_comedy_to_explain_science
Transcript not available
Current url: https://www.ted.com/talks/marian_wright_edelman_reflections_from_a_lifetime_fighting_to_end_child_poverty
Current url: https://www.ted.com/talks/melinda_babits_do_insects_have_personalities
Transcript not available
Current url: https://www.ted.com/talks/sean_davis_can_we_solve_global_warming_lessons_from_how_we_protected_the_ozone_layer
Current url: https://www.ted.com/talks/greta_thunberg_the_disarming_case_to_act_right_now_on_climate_change
Current url: https://www.ted.com/talks/daniel_finkel_can_you_solve_the_vampire_hunter_riddle
Current url: https://www.ted.com/talks/soraya_chemaly_the_power_of_women_s_anger
Current url: https://www.ted.com/t

Current url: https://www.ted.com/talks/li_wei_tan_the_fascinating_science_of_bubbles_from_soap_to_champagne
Current url: https://www.ted.com/talks/michael_c_bush_this_is_what_makes_employees_happy_at_work
Current url: https://www.ted.com/talks/katharine_hayhoe_the_most_important_thing_you_can_do_to_fight_climate_change_talk_about_it
Current url: https://www.ted.com/talks/paul_s_kindstedt_a_brie_f_history_of_cheese
Current url: https://www.ted.com/talks/madame_gandhi_and_amber_galloway_gallego_music_with_a_message_should_be_accessible
Current url: https://www.ted.com/talks/douglas_rushkoff_how_to_be_team_human_in_the_digital_future
Current url: https://www.ted.com/talks/carla_harris_how_to_find_the_person_who_can_help_you_get_ahead_at_work
Current url: https://www.ted.com/talks/nadjia_yousif_why_you_should_treat_the_tech_you_use_at_work_like_a_colleague
Current url: https://www.ted.com/talks/pete_buttigieg_how_one_mayor_forged_his_city_s_future_by_rethinking_its_past
Transcript not avai

Message: 

Current url: https://www.ted.com/talks/david_fleischer_how_to_fight_prejudice_through_policy_conversations
Transcript not available
Current url: https://www.ted.com/talks/finn_lutzow_holm_myrstad_how_tech_companies_deceive_you_into_giving_up_your_data_and_privacy
Current url: https://www.ted.com/talks/graham_allison_is_war_between_china_and_the_us_inevitable
Current url: https://www.ted.com/talks/keith_eggener_the_fascinating_history_of_cemeteries
Current url: https://www.ted.com/talks/joan_c_williams_we_won_t_fix_us_politics_until_we_talk_about_class
Message: 

Current url: https://www.ted.com/talks/fadi_chehade_and_bryn_freedman_what_everyday_citizens_can_do_to_claim_power_on_the_internet
Current url: https://www.ted.com/talks/brian_d_avery_how_rollercoasters_affect_your_body
Current url: https://www.ted.com/talks/lauren_esposito_what_scorpions_can_teach_us_about_brain_tumors
Transcript not available
Current url: https://www.ted.com/talks/rachel_wurzman_how_isolation_fuels

Current url: https://www.ted.com/talks/luhan_yang_how_to_create_a_world_where_no_one_dies_waiting_for_a_transplant
Current url: https://www.ted.com/talks/krishna_sudhir_how_do_cigarettes_affect_the_body
Current url: https://www.ted.com/talks/ghada_wali_how_i_m_using_lego_to_teach_arabic
Current url: https://www.ted.com/talks/bob_woodward_what_i_learned_investigating_nixon
Message: 

Current url: https://www.ted.com/talks/will_macaskill_what_are_the_most_important_moral_problems_of_our_time
Current url: https://www.ted.com/talks/elise_legrow_you_never_can_tell_over_the_mountain_across_the_sea
Current url: https://www.ted.com/talks/alan_lupack_is_there_any_truth_to_the_king_arthur_legends
Current url: https://www.ted.com/talks/john_bidden_not_about_me
Transcript not available
Current url: https://www.ted.com/talks/ruchi_shah_how_small_business_can_benefit_from_village_banking
Transcript not available
Current url: https://www.ted.com/talks/muslim_sahib_the_coming_out_beauty_419_sq_ft
Tran

Current url: https://www.ted.com/talks/christoph_niemann_you_are_fluent_in_this_language_and_don_t_even_know_it
Current url: https://www.ted.com/talks/ranveer_chandra_data_driven_farming_could_transform_agriculture
Transcript not available
Current url: https://www.ted.com/talks/bronwyn_king_you_may_be_accidentally_investing_in_cigarette_companies
Current url: https://www.ted.com/talks/janell_hobson_the_breathtaking_courage_of_harriet_tubman
Current url: https://www.ted.com/talks/supasorn_suwajanakorn_fake_videos_of_real_people_and_how_to_spot_them
Current url: https://www.ted.com/talks/christina_wallace_how_to_stop_swiping_and_find_your_person_on_dating_apps
Current url: https://www.ted.com/talks/ian_strange_home_is_where_the_art_is
Transcript not available
Current url: https://www.ted.com/talks/pratik_shah_how_ai_is_making_it_easier_to_diagnose_disease


In [ ]:
transcript_dataset = []
for o in my_tedx_list_final:
    for t in o['transcript']:
        transcript_dataset.append({"idx": o['idx'], "timestamp": t["timestamp"], "sentence": t["sentence"]})

In [ ]:
transcript_df = pd.DataFrame.from_dict(transcript_dataset)
transcript_df.to_csv('transcript_dataset.csv', index=False)